<a href="https://colab.research.google.com/github/DGuilherme/EnergyProductionScheduling/blob/main/PhaseONE/Experiments/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm Implmentation
*A stochastic global optimization algorithm*

# Imports


In [1]:
# install GA library
!pip install pygad

     |████████████████████████████████| 53 kB 1.4 MB/s 


In [7]:
import numpy
import pandas as pd
import pygad as ga
import copy
import time
import datetime
import sys
import random
import plotly.figure_factory as ff

# Datasets
*Note: the url for the dataset must be updated everytime this book is opened*

* [production_scheduling_maintenance_duedate_constraint](https://github.com/DGuilherme/EnergyProductionScheduling/blob/main/PhaseONE/Datasets/BJSP102_TestInstances/BJSP102_1.csv)
*   List item



## BJSP102 Dataset


In [ ]:

url = 'https://raw.githubusercontent.com/DGuilherme/EnergyProductionScheduling/main/PhaseONE/Datasets/BJSP102_TestInstances/BJSP102_1.csv?token=GHSAT0AAAAAABQ2Z462HLKR2BOJT5FHQGMEYPMGLJQ'
production_scheduling_maintenance_duedate_constraint = pd.read_csv(url)



In [ ]:
production_scheduling_maintenance_duedate_constraint.describe()


,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
count,40.000000,32.000000,16.000000,8.000000,8.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,9.957250,1.198438,9.585625,0.561250,3.695000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,10.398949,1.933157,8.646498,0.627886,2.742199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,1.000000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.000000,0.000000,4.057500,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4.000000,0.790000,6.715000,0.380000,3.980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,19.717500,1.010000,9.905000,1.107500,4.740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,30.020000,9.000000,29.130000,1.380000,9.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
production_scheduling_maintenance_duedate_constraint.head

<bound method NDFrame.head of                        Job ID     1  ... Unnamed: 19  Unnamed: 20
0      Type (job/maintenance)   job  ...         NaN          NaN
1                    Resource     9  ...         NaN          NaN
2             Processing time  5.36  ...         NaN          NaN
3    Due date (soft and hard)  61.3  ...         NaN          NaN
4                Release date     0  ...         NaN          NaN
..                        ...   ...  ...         ...          ...
264  Due date (soft and hard)   144  ...         NaN          NaN
265              Release date   144  ...         NaN          NaN
266            Preceding Jobs   NaN  ...         NaN          NaN
267           Precedence Type   NaN  ...         NaN          NaN
268                       NaN   NaN  ...         NaN          NaN

[269 rows x 21 columns]>

In [ ]:
# Remove nan values from pandas df
data = production_scheduling_maintenance_duedate_constraint.dropna(how='all', axis=1)
print(data)

                       Job ID     1  ... Unnamed: 6  Unnamed: 7
0      Type (job/maintenance)   job  ...        NaN         NaN
1                    Resource     9  ...        NaN         NaN
2             Processing time  5.36  ...        NaN         NaN
3    Due date (soft and hard)  61.3  ...        NaN         NaN
4                Release date     0  ...        NaN         NaN
..                        ...   ...  ...        ...         ...
264  Due date (soft and hard)   144  ...        NaN         NaN
265              Release date   144  ...        NaN         NaN
266            Preceding Jobs   NaN  ...        NaN         NaN
267           Precedence Type   NaN  ...        NaN         NaN
268                       NaN   NaN  ...        NaN         NaN

[269 rows x 8 columns]


## la02 Dataset


In [3]:
times = [[20, 87, 31, 76, 17], [25, 32, 24, 18, 81], [72, 23, 28, 58, 99], [86, 76, 97, 45, 90], [27, 42, 48, 17, 46], [67, 98, 48, 27, 62], [28, 12, 19, 80, 50], [63, 94, 98, 50, 80], [14, 75, 50, 41, 55], [72, 18, 37, 79, 61]]
machines = [[0, 3, 1, 4, 2], [4, 2, 0, 1, 3], [1, 2, 4, 0, 3], [2, 1, 4, 0, 3], [4, 0, 3, 2, 1], [1, 0, 4, 3, 2], [4, 1, 3, 0, 2], [1, 0, 2, 3, 4], [4, 0, 2, 1, 3], [4, 2, 1, 3, 0]]
n = 10

# Genetic Algorithm Functions


## Evolution 

In [8]:
#During the crossover we select gens from the father from the start to the end index defined, we remove those from the mother
#Then we add them to the resultant in the same order that it was in the father origininally
def crossover(father, mother, start_index, end_index):
    father_gen = father[0][start_index:end_index]
    fetus = removeFromList(mother[0], father_gen)
    result = []
    result.extend(fetus[:start_index])
    result.extend(father_gen)
    result.extend(fetus[start_index:])
    return [result, None]

def removeFromList(parent, list):
    seen = set()
    seen_add = seen.add
    return [x for x in parent if not (x in list or seen_add(x))]

#mutate one member of the poupulation randomly excluding the first one(best individual)
#We just change the order of the permutation by one
def mutation(population, mutation_rate):
    if(random.random() > mutation_rate):
        candidate = random.choice(population[1:])
        swap_rnd(candidate[0])
        candidate[1] = None


def swap_rnd(config):
    id1 = random.choice(range(len(config)))
    id2 = random.choice(range(len(config)))
    tmp = config[id1]
    config[id1] = config[id2]
    config[id2] = tmp
    return config



def evolve(population, mutation_rate):
    #Important: the population should be sorted before evolve

    #We delete the worst individual of the population
    population.pop()

    #we choose a mother and father for the new individual
    father = random.choice(population)
    mother = random.choice(population)
    while(mother == father):
        mother = random.choice(population)
    
    indexes = range(len(father[0]))

    #we select wich part of the father will go to the mother
    start_index = random.choice(indexes)
    end_index = random.choice(indexes[start_index:])

    #we generate the baby with the crossover
    baby = crossover(father, mother, start_index, end_index)

    #we add the new member to the population
    population.append(baby)

    #we trigger the mutation for one of the population, depending on the mutation rate
    mutation(population, mutation_rate)
    return population



def fromPermutation(permutation, n):
    return list(map(lambda  x: x%n, permutation))

### Population

In [9]:
#We generate the number of population
def generate_population(number, n, mn):
        population = []
        for i in range(number):
            addRandomIndividual(population, n, mn)
        return population


#Each individual is compose by a permutation(list from 0 to the job_number*machine_number)
#And a second parameter that is filled with the result of the makespan for the permutation
#We keep track of the result to not calculate multiple times the same result unnecesarily
#Is important to remove that number every time the permutation change
def addRandomIndividual(population, n, mn):
    ind = list(range(n*mn))
    random.shuffle(ind)
    population.append([ind, None])


def checkDiversity(population, diff, n, mn):
    if diff < 0.03:
        replaceWithRandomPopulation(population, int(n/3), n, mn)
    if diff < 0.05:
        replaceWithRandomPopulation(population, int(n/5), n, mn)
    elif diff < 0.1:
        replaceWithRandomPopulation(population, int(n/10), n, mn)

def replaceWithRandomPopulation(population, q, n, mn):
    for i in range(q):
        population.pop()
    for i in range(q):
        addRandomIndividual(population, n, mn)

### Fitness Function


In [21]:

def calculateMakespan(times, machines, config, n):
    time_table = []
    times = copy.deepcopy(times)
    machines = copy.deepcopy(machines)
    mn = len(machines[0])
    for i in range(mn):
        time_table.append([])

    current_times = [0]*n
    total_time = 0
    for j in config:
        job = j%n
        current_machine = machines[job].pop(0)
        current_time = current_times[job]
        machine_usage = time_table[current_machine]
        usage_time = times[job].pop(0)
        current_time, total_time = fillTimeSlot(machine_usage, current_time, usage_time, job, total_time)

        current_times[job] = current_time

    return total_time, 

def fillTimeSlot(machine_usage, current_time, usage_time, job, total_time):
    if len(machine_usage) > 0:
        prev = 0
        slot = None
        for used_slots in machine_usage:
            start = used_slots[0]
            end = used_slots[1]

            if start < current_time and current_time < end:
                current_time = end

            if prev == 0 and start > current_time + usage_time:
                slot = [current_time, usage_time + current_time, job]
                break
            elif start - prev >= usage_time and current_time <= prev:
                slot = [current_time, current_time + usage_time, job]
                break

            prev = end
            if end > current_time:
                current_time = end

        if slot == None:
            slot = [current_time, current_time + usage_time, job]

        current_time = slot[1]
        machine_usage.append(slot)
        machine_usage.sort(key=lambda x: x[1])

        if slot[1] > total_time:
            total_time = slot[1]

    else: 
        machine_usage.append([current_time, usage_time + current_time, job])
        current_time += usage_time

    return current_time, total_time

def sortAndGetBestIndividual(population):
        best_individual = None
        best_result = None
        for individual in population:
            result = None
            if individual[1] != None: 
                result, table = calculateMakespan(times, machines, individual[0], n)
                individual[1] = result
            else: 
                result = individual[1]

            if not best_result or result < best_result:
                best_result = result
                best_individual = individual

        population.sort(key=lambda x: x[1])
        return best_individual, best_result


  
def getFitness(population):
    prev = population[0][1]
    total = 0
    diffPercentage = 0.0
    for ind in population:
        curr = ind[1]
        total += curr
        diffPercentage += (curr/float(prev)) - 1
        prev = curr

    return total, diffPercentage



def printProgress(bestValue, iterations, timeElapsed):
    sys.stdout.write("\rIterations: {0} | Best result found {1} | Time elapsed: {2}s".format(iterations, bestValue, int(timeElapsed)))
    sys.stdout.flush()

### Utils


In [20]:
def printTable(table):
    i = 1
    print("TABLE: ")
    for row in table:
        print("M%s: %s" %(i, row))
        i += 1

def plotResult(table, maxValue):
    df = []
    mn = 0
    colors = []
    for row in table:
        mn += 1
        row.sort(key=lambda x: x[2])
        for slot in row:
            start_time=str(datetime.timedelta(seconds=slot[0]))
            end_time=str(datetime.timedelta(seconds=slot[1]))
            today = datetime.date.today()
            entry = dict(
                Task='Machine-{0}'.format(mn), 
                Start="{0} {1}".format(today, start_time), 
                Finish="{0} {1}".format(today, end_time),
                duration=slot[1] - slot[0],
                Resource='Job {0}'.format(slot[2] + 1)
                )
            df.append(entry)

            #Generate random colors
            if(len(colors) < len(row)):
                a = min(255 - ( slot[2] * 10 ), 255)
                b = min(slot[2] * 10, 255)
                c = min(255, int(random.random() * 255))
                colors.append("rgb({0}, {1}, {2})".format(a, b, c))

    #In order to see the line ordered by integers and not by dates we need to generate the dateticks manually
    #we create 11 linespaced numbers between 0 and the maximum value
    num_tick_labels = numpy.linspace(start = 0, stop = maxValue, num = 11, dtype = int)
    date_ticks = ["{0} {1}".format(today, str(datetime.timedelta(seconds=int(x)))) for x in num_tick_labels]

    fig = ff.create_gantt(df,colors=colors, index_col='Resource', group_tasks=True, show_colorbar=True, showgrid_x=True, title='Job shop Schedule')
    fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
    fig.show()

### Main

In [18]:
def genetic(times, machines, n, population_number, iterations, rate, target):
    machine_number = len(machines[0])
    start_time = time.time()

    print(population_number)
    print(n)
    print(machine_number)

    population = generate_population(population_number, n, machine_number)

    print(population)
    
    global_best_ind, global_best = sortAndGetBestIndividual(population)
    
    ##if we don't define a target we set the number of iterations we want 
    if not target:
        for i in range(iterations):
            population = evolve(population, rate)
            best_ind, best_result = sortAndGetBestIndividual(population)
            total_fitness, diffPercentage = getFitness(population)

            if(not global_best or best_result < global_best):
                global_best = best_result
                global_best_ind = copy.deepcopy(best_ind)

            printProgress(best_result, i, time.time() - start_time)
            checkDiversity(population, diffPercentage, n, machine_number)
    else:
        #If we define a target we iterate until the best result reach that target
        i = 0
        while(target < global_best):
            i += 1
            #in every iteration: 
            #We evolve the population
            population = evolve(population, rate)
            #We find the best individual 
            best_ind, best_result = sortAndGetBestIndividual(population)
            #We calculate the diversity % between the population and the total_fitness(sum of all the results)
            total_fitness, diffPercentage = getFitness(population)

            #if the result found is better than the global found we update the global
            if(not global_best or best_result < global_best):
                global_best = best_result
                global_best_ind = copy.deepcopy(best_ind)
            #We print the progress so far and the time elapsed
            printProgress(best_result, i, time.time() - start_time)
            #We check the diversity, in case the diversity percentage is very low we delete a number of the population and we add randome members
            checkDiversity(population, diffPercentage, n, machine_number)

    
    best_result, best_table = calculateMakespan(times, machines, global_best_ind[0], n)           
    print("\nOVERALL RESULT")
    print("RESULT: %s" %best_result)                 
    print('the elapsed time:%ss'% (int(time.time() - start_time)))
    print("Permutation: ")
    print(fromPermutation(global_best_ind[0], n))
    printTable(best_table)
    plotResult(best_table, best_result)

# Results

What kind of methods can be used to evaluate the results from the algorithm


In [22]:
#@title Insert here the desired parameters

times = [[20, 87, 31, 76, 17], [25, 32, 24, 18, 81], [72, 23, 28, 58, 99], [86, 76, 97, 45, 90], [27, 42, 48, 17, 46], [67, 98, 48, 27, 62], [28, 12, 19, 80, 50], [63, 94, 98, 50, 80], [14, 75, 50, 41, 55], [72, 18, 37, 79, 61]]
machines = [[0, 3, 1, 4, 2], [4, 2, 0, 1, 3], [1, 2, 4, 0, 3], [2, 1, 4, 0, 3], [4, 0, 3, 2, 1], [1, 0, 4, 3, 2], [4, 1, 3, 0, 2], [1, 0, 2, 3, 4], [4, 0, 2, 1, 3], [4, 2, 1, 3, 0]]
n = 10

# the required functions
target = None

population_size = 10 #@param {type:"number"}
mutation_rate= 0.2 #@param {type:"number"}
iterations= 20 #@param {type:"number"}

# main
genetic(times, machines, n, population_size, iterations, mutation_rate, target)


10
10
5
[[[24, 28, 1, 19, 42, 0, 26, 11, 45, 9, 3, 44, 31, 2, 14, 37, 41, 48, 4, 20, 13, 33, 39, 23, 29, 46, 25, 21, 8, 12, 7, 27, 17, 43, 16, 34, 6, 47, 49, 15, 22, 30, 38, 18, 32, 40, 35, 36, 10, 5], None], [[7, 41, 1, 6, 0, 20, 11, 42, 19, 44, 36, 27, 3, 22, 8, 46, 17, 34, 30, 47, 9, 4, 12, 35, 14, 40, 33, 31, 13, 28, 10, 24, 2, 16, 38, 23, 45, 15, 26, 37, 29, 43, 5, 32, 18, 49, 48, 21, 25, 39], None], [[21, 7, 2, 29, 40, 5, 10, 46, 1, 33, 26, 20, 18, 30, 45, 11, 8, 43, 13, 12, 24, 34, 17, 36, 44, 16, 9, 19, 35, 31, 47, 14, 0, 4, 37, 27, 49, 6, 25, 41, 39, 42, 38, 3, 32, 23, 28, 48, 15, 22], None], [[20, 41, 49, 32, 29, 22, 17, 11, 7, 14, 5, 36, 47, 38, 25, 35, 40, 26, 16, 28, 2, 30, 9, 15, 4, 37, 0, 8, 18, 13, 12, 34, 42, 21, 23, 46, 31, 48, 44, 24, 39, 3, 6, 27, 1, 45, 19, 33, 43, 10], None], [[19, 12, 49, 24, 45, 17, 30, 48, 11, 47, 37, 10, 16, 41, 44, 32, 20, 9, 3, 23, 18, 1, 38, 14, 42, 6, 43, 39, 22, 31, 28, 0, 7, 34, 4, 33, 8, 46, 29, 5, 40, 15, 27, 25, 36, 35, 2, 21, 13, 26]

TypeError: ignored

NameError: ignored